In [62]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

db = DocBin()

In [63]:
!python -m spacy download en_core_web_lg

/Users/sofiahalima/PycharmProjects/EntityExtractionDemo/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 7.3 MB/s eta 0:00:0000:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [64]:
nlp=spacy.load('en_core_web_lg')
from spacy.lang.en import English

In [65]:
file = open('/Users/sofiahalima/PycharmProjects/EntityExtractionDemo/data/annotations.json')
training_data = json.load(file)

In [66]:
for text,annotation in tqdm(training_data["annotations"]):
    doc = nlp.make_doc(text)
    entities=[]
    for start, end, label in annotation["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is not None:
            entities.append(span)
            
    doc.ents = entities
    db.add(doc)
    
    db.to_disk("training_data_doc.spacy")

100%|██████████| 24/24 [00:00<00:00, 713.94it/s]


In [68]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency --force

/Users/sofiahalima/PycharmProjects/EntityExtractionDemo/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [69]:
!python -m spacy train config.cfg --output output --paths.train training_data_doc.spacy --paths.dev training_data_doc.spacy

/Users/sofiahalima/PycharmProjects/EntityExtractionDemo/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     53.61    0.00    0.00    0.00    0.00
 28     200        435.98   2116.20  100.00  100.00  100.00    1.00
 62     400          0.03      0.03  100.00  100.00  100.00    1.00
105     600          3.09      5.23  100.00  100.00  100.00    1.0

In [70]:
nlp_news = spacy.load('output/model-best')

In [71]:
doc = nlp_news(''' Vladimir Putin has led Russia's Victory Day commemorations with a parade in Red Square and heightened security after days of Ukrainian strikes targeting the capital.

China's Xi Jinping joined Putin as the Russian president told thousands of soldiers and more than 20 international leaders that Russia remembered the lessons of World War Two.

Putin used his speech to tie the war to today's full-scale invasion of Ukraine, and said all of Russia was behind what he called the "special military operation" - now well into its fourth year.

For the first time, a column of trucks carrying various combat drones took part in the Victory Day parade, apparently because of their widescale use in Ukraine.''')

In [73]:
result =(spacy.displacy.render(doc, style="ent"))